This notebook is part of [miubrt](https://github.com/meteo-ubonn/miubrt).

Copyright (c) [miubrt developers](https://github.com/meteo-ubonn/miubrt/blob/main/CONTRIBUTORS.txt). 
Distributed under the MIT License. See [LICENSE.txt](https://github.com/meteo-ubonn/miubrt/blob/main/LICENSE.txt) for more info.

# Create NetCDF4 output - Explore BoXPol/JuXPol
<div class="alert alert-info">

**User Info** <br>
 Global Data - User Input 
 
 start_time, end_time, location and sweep
 
 location = "BoXPol" or "JuXPol"  (X-band radar)
 
 elev_id  = 0 ... 9   (IDs for elevation angles)
    
 For BoXPol, the elevation folders name change in 2017 (check for error message)
   
</div>
 

In [1]:
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import datetime as dt
import numpy as np
import dask
import wradlib as wrl
import xarray as xr

In [3]:
# for using local development version, uncomment next lines otherwise installed `miubrt` version will be used
miubrt_dir = "../../"
if sys.path[0] != miubrt_dir:
     sys.path.insert(0, "../../")

In [4]:
import miubrt as mrt

# Global Data - USER INPUT

In [10]:
start_time = dt.datetime(2015, 7, 5, 14, 0, 0)
end_time = dt.datetime(2015, 7, 5, 17, 30, 0)
location = "JuXPol"
elev_id  = 1
# User Input End #

if location == "BoXPol":
    snames = ["n_ppi_010deg","ppi_1p5deg","ppi_2p4deg","ppi_3p4deg","n_ppi_045deg","n_ppi_082deg", 
          "n_ppi_110deg","n_ppi_140deg","n_ppi_180deg","n_ppi_280deg"]
    # 2017 onwards, elevations changed ?
    #snames = ["n_ppi_010deg","n_ppi_020deg","n_ppi_031deg","n_ppi_045deg","n_ppi_060deg","n_ppi_082deg", 
    #      "n_ppi_110deg","n_ppi_140deg", "n_ppi_180deg", "n_ppi_280deg"]
    #
    sname  = snames[elev_id]
    sweep  = 0
elif location == "JuXPol":
    sname = "DWD_Vol_2"
    sweep = elev_id

output_path = ""

# Create Netcdf Dataset of wanted radar data

In [9]:
fname_raw = mrt.io.create_netcdf_dataset(location, sname, start_time, end_time, sweep, 
                                         outpath=output_path, engine='h5netcdf', chunks=None)
fname_raw

Collecting: 100%|██████████| 42/42 [00:02<00:00, 18.36 Timesteps/s]


[########################################] | 100% Completed |  0.1s


'/data/hubhome/pshrestha/miubrt/notebooks/getting_started/JuXPol_280_20150705_1400_1730.nc'

# Load and Explore Dataset

In [29]:
import matplotlib as mpl
import hvplot
import hvplot.xarray
import holoviews as hv

## Load

In [30]:
fname_raw = '/data/hubhome/pshrestha/miubrt/notebooks/getting_started/BoXPol_10_20170706_0000_0000.nc'
swp = xr.open_dataset(fname_raw, chunks={"time": 12})

## Georeference

In [31]:
swp = swp.pipe(wrl.georef.georeference_dataset)

## Interactive Exploration

In [32]:
swp = swp.chunk()

In [33]:
dbz_plot = swp.hvplot.quadmesh(groupby='time',
                               x='x', y='y', 
                               z='DBZH', 
                               rasterize=True, 
                               clim=(-20,50), 
                               cmap=mpl.cm.nipy_spectral,
                               frame_width=500,
                               aspect=1,
                              )

dbz_plot = dbz_plot.options(color_levels=list(np.arange(-20, 51, 2)))
dbz_plot

:DynamicMap   [time]
   :Image   [x,y]   (DBZH)